In [1]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
import torch_geometric
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

/opt/homebrew/anaconda3/envs/geometric/lib/python3.12/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/opt/homebrew/anaconda3/envs/geometric/lib/python3.12/site-packages/libpyg.so, 0x0006): Library not loaded: /Library/Frameworks/Python.framework/Versions/3.12/Python
  Referenced from: <3F6503CE-E988-3D6E-BA33-026DFC13E8C5> /opt/homebrew/anaconda3/envs/geometric/lib/python3.12/site-packages/libpyg.so
  Reason: tried: '/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file), '/Library/Frameworks/Python.framework/Versions/3.12/Python' (no such file)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/opt/homebrew/anaconda3/envs/geometric/lib/python3.12/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-spar

In [58]:
sample_size = 6000

In [59]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT-base
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

bert_model.eval()  # Evaluation mode (no dropout)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [2]:
data = pd.read_csv('final_fantasy.csv')

In [3]:
data.head()

,Unnamed: 0,user_steam_id,user_playtime_forever,user_playtime_2weeks,user_has_leaderboards,user_has_community_visible_stats,user_content_descriptorids,game_appid,game_name,game_developer,...,game_price,game_initialprice,game_discount,game_ccu,game_owners_estimate,game_review_ratio,genres_en,categories_en,short_description_en,detailed_description_en
0,0,76561198060785055,7279,0.0,False,True,"[2, 5]",240,Counter-Strike: Source,Valve,...,9.99,9.99,0.0,15002,10000000,0.962647,['Action'],"['Multi-Player', 'Cross-Platform Multiplayer',...",Counter-Strike: Source blends Counter-Strike's...,THE NEXT INSTALLMENT OF THE WORLD'S # 1 ONLINE...
1,1,76561198060785055,0,0.0,False,True,[0],12210,Grand Theft Auto IV: The Complete Edition,"Rockstar North, Rockstar Toronto",...,19.99,19.99,0.0,3464,5000000,0.823607,"['Action', 'Adventure']","['Single-Player', 'Multi-Player', 'Partial Con...","Niko Bellic, Johnny Klebitz and Luis Lopez all...",Important Updates To Grand Theft Auto IV and E...
2,2,76561198060785055,717,0.0,True,True,[0],620,Portal 2,Valve,...,9.99,9.99,0.0,3194,10000000,0.986921,"['Action', 'Adventure']","['Single Person', 'Multiple People', 'Cooperat...",The Lifetime Test Plan is now upgraded and you...,Portal 2 creates another successor to the gran...
3,3,76561198060785055,4578,0.0,False,True,[0],105600,Terraria,Re-Logic,...,9.99,9.99,0.0,38390,20000000,0.974917,"['Action', 'Adventure', 'Indie', 'Rpg']","['Single-Player', 'Multi-Player', 'Pvp', 'Onli...","Dig, fight, explore, build! Nothing is impossi...","Dig, Fight, Explore, Build: The very world is ..."
4,4,76561198060785055,0,0.0,True,True,[0],46520,Wasteland Angel,Octane Games,...,4.99,4.99,0.0,0,50000,0.587302,"['Action', 'Indie']","['Single-Player', 'Steam Achievements', 'Parti...",Wasteland Angel is a top-down twin-stick shoot...,Bummer! World War III happened and killed most...


In [4]:
def create_game_mapper(df):
    # Convert the two columns into a dictionary
    game_mapper = dict(zip(df['game_appid'], df['game_name']))
    return game_mapper

# Example usage
# Assuming you have a DataFrame named 'df' with columns 'game_appid' and 'game_name'
game_mapper = create_game_mapper(data)
game_mapper[240]

'Counter-Strike: Source'

Feature Embeddings:
- Start off with pure genre if it is good enough
- (3 approaches) Eval pure genre approach with pure short descriptons embeddings then try combining

In [5]:
data['text_for_embedding'] = ( data['short_description_en'].fillna('') + ' ' +  data['genres_en'].astype(str) )

Short Description + Genre == Token Size

In [6]:
word_count = 0
for text in data['text_for_embedding']:
    word_count+=len(text)
print(f'Total number of tokens: {word_count}')

Total number of tokens: 472731179


install transformer of choice (BeRT)

In [44]:
def embed_text(texts, tokenizer, model, max_length=128):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length, padding='max_length')
        with torch.no_grad():
            outputs = model(**inputs)
        # CLS pooling (using [CLS] token embedding)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        embeddings.append(cls_embedding.squeeze(0))
    embeddings = torch.stack(embeddings)
    return embeddings

In [ ]:
# game_embeddings = embed_text(data['text_for_embedding'].tolist()[:sample_size], tokenizer, bert_model)
# print(game_embeddings.shape)

KeyError: 'text_for_embedding'

## User/Game Node Mapping

### how to dedupe interactions
check composite keys (user/game), if all 1s then only unique interactions

In [ ]:
data.groupby(by=['user_steam_id','game_appid']).count().reset_index()['user_playtime_forever'].value_counts()

user_playtime_forever
1    1996413
Name: count, dtype: int64

### init mappings

In [31]:
# Unique users and games
unique_users = data['user_steam_id'].unique()
unique_games = data['game_appid'].unique()

# Build mappings
user_id_map = {user_id: idx for idx, user_id in enumerate(unique_users)}
game_id_map = {game_id: idx + len(unique_users) for idx, game_id in enumerate(unique_games)}

# How many nodes?
num_users = len(user_id_map)
num_games = len(game_id_map)
num_nodes = num_users + num_games

In [32]:
num_users, num_games, num_nodes

(5842, 31397, 37239)

## Building out user/game nodes
- Approximately takes an hour to run, so we will have preloaded from our sample dataset.

In [ ]:
# build edges, load a new user to recommend games (edges user to game)
user_nodes = []
game_nodes = []
for idx, row in tqdm(data.iterrows(), desc="Processing rows"):
    # get indices and append them
    user_idx = user_id_map[row['user_steam_id']]
    game_idx = game_id_map[row['game_appid']]
    # print(user_idx, game_idx)
    user_nodes.append(user_idx)
    game_nodes.append(game_idx)
    # print(f'User Node {user_idx} and Game Node {game_idx} were appended')
    if idx == sample_size:
        break

Processing rows: 2000it [00:00, 2269.01it/s]


### Array Dimension Validation
- Also upon running initial mapping, storing list in .npy file

In [34]:
len(game_nodes)==len(user_nodes)

True

In [35]:
edge_index = torch.tensor([user_nodes, game_nodes], dtype=torch.long)

In [ ]:
user_features = np.zeros((num_users, 768))  # dummy features for users
game_features = game_embeddings.numpy()      # real features for games

# Stack vertically: first users, then games
X = np.vstack([user_features, game_features])
X = torch.tensor(X, dtype=torch.float)
print(X.shape)  # (num_nodes, 768)

torch.Size([7842, 768])


## attribute selection for edges

if we want more features, add here

In [ ]:
edge_attrs = []

for idx, row in tqdm(data.iterrows(), desc="Processing rows"):
    playtime = row['user_playtime_forever']
    edge_attrs.append([playtime])  # wrapping in list for 1-dim feature

edge_attr = torch.tensor(edge_attrs, dtype=torch.float)
print(edge_attr.shape)  # (num_edges, 1)

Processing rows: 1996413it [00:24, 81540.41it/s]


torch.Size([1996413, 1])


create data for geometric graph

In [38]:
from torch_geometric.data import Data

graph = Data(
    x=X,
    edge_index=edge_index,
    edge_y=edge_attr
)

print(graph)

Data(x=[7842, 768], edge_index=[2, 2001], edge_y=[1996413, 1])


test/train/val split

In [39]:
from torch_geometric.transforms import RandomLinkSplit

transform = RandomLinkSplit(
    num_val=0.05,        # 5% for validation
    num_test=0.10,       # 10% for testing
    is_undirected=True,  # True if your graph is undirected (user <-> game is undirected)
    add_negative_train_samples=False  # We will do negative sampling manually during training
)

# Apply the transform
train_data, val_data, test_data = transform(graph)

print(train_data)
print(val_data)
print(test_data)

Data(x=[7842, 768], edge_index=[2, 3402], edge_y=[1996413, 1], edge_label=[1701], edge_label_index=[2, 1701])
Data(x=[7842, 768], edge_index=[2, 3402], edge_y=[1996413, 1], edge_label=[200], edge_label_index=[2, 200])
Data(x=[7842, 768], edge_index=[2, 3602], edge_y=[1996413, 1], edge_label=[400], edge_label_index=[2, 400])


Encoder

In [40]:
from torch_geometric.nn import SAGEConv
import torch.nn as nn
import torch.nn.functional as F

In [41]:
class GNNEncoder(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
    def forward(self, X, edge_index):
        X = self.conv1(X, edge_index)
        X = F.relu(X)
        X = self.conv2(X, edge_index)
        return X

In [42]:
class LinkPredictor(nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin = nn.Linear(hidden_channels*2, 1)
    def forward(self, i, j):
        X = torch.cat([i,j], dim=1)
        return torch.sigmoid(self.lin(X))

In [43]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'mps')

hidden_channels = 128
encoder = GNNEncoder(in_channels=graph.num_node_features, hidden_channels=hidden_channels).to(device)
predictor = LinkPredictor(hidden_channels=hidden_channels).to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(predictor.parameters()), lr=0.001)

train_data = train_data.to(device)
val_data = val_data.to(device)
test_data = test_data.to(device)

In [44]:
test_data

Data(x=[7842, 768], edge_index=[2, 3602], edge_y=[1996413, 1], edge_label=[400], edge_label_index=[2, 400])

In [45]:
def train():
    encoder.train()
    predictor.train()
    optimizer.zero_grad()
    M = encoder(train_data.x, train_data.edge_index)
    preds = predictor(M[train_data.edge_label_index[0]], M[train_data.edge_label_index[1]])
    loss = F.binary_cross_entropy(preds.squeeze(), train_data.edge_label.float())
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def test(test_data):
    encoder.eval()
    predictor.eval()
    M = encoder(train_data.x, train_data.edge_index)
    preds = predictor(
        M[test_data.edge_label_index[0]], 
        M[test_data.edge_label_index[1]]
    )
    labels = test_data.edge_label.float()

    pred_labels = (preds.squeeze() > 0.5).float()

    acc = (pred_labels == labels).sum().item() / labels.size(0)
    return acc

In [46]:
epochs = 50

In [47]:
for epoch in tqdm(range(1, epochs + 1)):
    loss = train()
    val_acc = test(val_data)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}')

test_acc = test(test_data)
print(f'\nTest Accuracy: {test_acc:.4f}')

 30%|███       | 15/50 [00:03<00:05,  6.88it/s]

Epoch 01, Loss: 0.6585, Val Acc: 0.5000
Epoch 02, Loss: 0.3596, Val Acc: 0.5000
Epoch 03, Loss: 0.1637, Val Acc: 0.5000
Epoch 04, Loss: 0.0548, Val Acc: 0.5000
Epoch 05, Loss: 0.0145, Val Acc: 0.5000
Epoch 06, Loss: 0.0035, Val Acc: 0.5000
Epoch 07, Loss: 0.0008, Val Acc: 0.5000
Epoch 08, Loss: 0.0002, Val Acc: 0.5000
Epoch 09, Loss: 0.0001, Val Acc: 0.5000
Epoch 10, Loss: 0.0000, Val Acc: 0.5000
Epoch 11, Loss: 0.0000, Val Acc: 0.5000
Epoch 12, Loss: 0.0000, Val Acc: 0.5000
Epoch 13, Loss: 0.0000, Val Acc: 0.5000
Epoch 14, Loss: 0.0000, Val Acc: 0.5000
Epoch 15, Loss: 0.0000, Val Acc: 0.5000
Epoch 16, Loss: 0.0000, Val Acc: 0.5000
Epoch 17, Loss: 0.0000, Val Acc: 0.5000
Epoch 18, Loss: 0.0000, Val Acc: 0.5000
Epoch 19, Loss: 0.0000, Val Acc: 0.5000
Epoch 20, Loss: 0.0000, Val Acc: 0.5000
Epoch 21, Loss: 0.0000, Val Acc: 0.5000
Epoch 22, Loss: 0.0000, Val Acc: 0.5000
Epoch 23, Loss: 0.0000, Val Acc: 0.5000
Epoch 24, Loss: 0.0000, Val Acc: 0.5000
Epoch 25, Loss: 0.0000, Val Acc: 0.5000


100%|██████████| 50/50 [00:03<00:00, 15.48it/s]

Epoch 31, Loss: 0.0000, Val Acc: 0.5000
Epoch 32, Loss: 0.0000, Val Acc: 0.5000
Epoch 33, Loss: 0.0000, Val Acc: 0.5000
Epoch 34, Loss: 0.0000, Val Acc: 0.5000
Epoch 35, Loss: 0.0000, Val Acc: 0.5000
Epoch 36, Loss: 0.0000, Val Acc: 0.5000
Epoch 37, Loss: 0.0000, Val Acc: 0.5000
Epoch 38, Loss: 0.0000, Val Acc: 0.5000
Epoch 39, Loss: 0.0000, Val Acc: 0.5000
Epoch 40, Loss: 0.0000, Val Acc: 0.5000
Epoch 41, Loss: 0.0000, Val Acc: 0.5000
Epoch 42, Loss: 0.0000, Val Acc: 0.5000
Epoch 43, Loss: 0.0000, Val Acc: 0.5000
Epoch 44, Loss: 0.0000, Val Acc: 0.5000
Epoch 45, Loss: 0.0000, Val Acc: 0.5000
Epoch 46, Loss: 0.0000, Val Acc: 0.5000
Epoch 47, Loss: 0.0000, Val Acc: 0.5000
Epoch 48, Loss: 0.0000, Val Acc: 0.5000
Epoch 49, Loss: 0.0000, Val Acc: 0.5000
Epoch 50, Loss: 0.0000, Val Acc: 0.5000

Test Accuracy: 0.5000


In [48]:
@torch.no_grad()
def recommend_new_games_for_user(user_idx, train_data, top_k=5):
    encoder.eval()
    predictor.eval()

    # Step 1: Get node embeddings
    z = encoder(train_data.x, train_data.edge_index)

    user_embeddings = z[:num_users]
    game_embeddings = z[num_users:]

    user_emb = user_embeddings[user_idx].unsqueeze(0)
    user_emb_expanded = user_emb.expand(game_embeddings.size(0), -1)

    # Step 2: Predict scores for all games
    scores = predictor(user_emb_expanded, game_embeddings).squeeze()

    # Step 3: Find already played games
    # Find edges where source == user_idx
    user_edges = train_data.edge_index[0] == user_idx
    played_game_indices = train_data.edge_index[1][user_edges] - num_users  # get game indices

    played_game_indices = played_game_indices.tolist()

    # Step 4: Mask scores of already played games
    scores[played_game_indices] = -1e9  # set played games' scores very low so they won't be selected

    # Step 5: Pick top-k games
    top_scores, top_game_indices = torch.topk(scores, k=top_k)

    recommendations = []
    for i, game_idx in enumerate(top_game_indices):
        game_id = unique_games[game_idx.item()]  # map back to real game ID
        score = top_scores[i].item()
        recommendations.append((game_id, score))
    return recommendations

In [ ]:
user_idx = 0
top_k = 5
recommendations = recommend_new_games_for_user(user_idx=user_idx, train_data=train_data, top_k=top_k)

print(f"\nTop {top_k} new game recommendations for User {user_idx}:")
for rank, (game_id, score) in enumerate(recommendations, start=1):
    print(f"Rank {rank}: Game ID = {game_mapper[game_id]}, Score = {score:.4f}")


Top 5 new game recommendations for User 0:
Rank 1: Game ID = Portal 2, Score = 1.0000
Rank 2: Game ID = Path of Exile, Score = 1.0000
Rank 3: Game ID = Cubetractor, Score = 1.0000
Rank 4: Game ID = The Forest, Score = 1.0000
Rank 5: Game ID = Rocket League, Score = 1.0000


In [202]:
# from torch_geometric.utils import to_networkx
# import networkx as nx

# # Convert to NetworkX graph
# G = to_networkx(data, to_undirected=True)

# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 12))
# pos = nx.spring_layout(G, seed=42)  # spring layout (force-directed)

# nx.draw(
#     G,
#     pos,
#     node_size=10, 
#     edge_color="gray", 
#     alpha=0.7,
#     node_color="blue"
# )
# plt.title("User-Game Bipartite Graph Visualization")
# plt.show()

Actionable Steps

# Cold Start Recommendation if the user is not in the graph

In [67]:
import requests

STEAM_API_KEY = os.getenv("STEAM_API_KEY")

# 👇 Build this once when you load your graph
game_idx_map = {appid: idx for idx, appid in enumerate(unique_games)}

@torch.no_grad()
def recommend_games_by_steam_id(steam_id, user_id_map, train_data, top_k=5):
    encoder.eval()
    predictor.eval()

    z = encoder(train_data.x, train_data.edge_index)

    user_embeddings = z[:num_users]
    game_embeddings = z[num_users:]

    if steam_id in user_id_map:
        print(f"Steam ID {steam_id} found in graph ✅")
        user_idx = user_id_map[steam_id]
        user_emb = user_embeddings[user_idx].unsqueeze(0)
    else:
        print(f"Steam ID {steam_id} not found. Trying Steam API cold-start fallback...")

        owned_game_ids = get_owned_games_from_steam(steam_id)
        if not owned_game_ids:
            print(f"No games found for Steam ID {steam_id} Cannot recommend.")
            return []

        known_owned_games = [appid for appid in owned_game_ids if appid in unique_games]
        if not known_owned_games:
            print(f"No known games found for Steam ID {steam_id} Cannot recommend.")
            return []

        game_idxs = [game_id_map[appid] - num_users for appid in known_owned_games]
        user_emb = game_embeddings[game_idxs].mean(dim=0).unsqueeze(0)
        print(f"Created cold-start embedding from {len(game_idxs)} owned games")

    user_emb_expanded = user_emb.expand(game_embeddings.size(0), -1)
    scores = predictor(user_emb_expanded, game_embeddings).squeeze()

    if steam_id in user_id_map:
        user_idx = user_id_map[steam_id]
        user_edges = train_data.edge_index[0] == user_idx
        played_game_indices = train_data.edge_index[1][user_edges] - num_users
        played_game_indices = played_game_indices.tolist()
        scores[played_game_indices] = -1e9
    else:
        played_game_indices = [game_idx_map[appid] for appid in known_owned_games if appid in game_idx_map]
        scores[played_game_indices] = -1e9

    top_scores, top_game_indices = torch.topk(scores, k=top_k)

    recommendations = []
    for i, game_idx in enumerate(top_game_indices):
        game_id = unique_games[game_idx.item()]
        score = top_scores[i].item()
        recommendations.append((game_id, score))

    return recommendations

# --- Helper ---
def get_owned_games_from_steam(steam_id):
    url = "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/"
    params = {
        'key': STEAM_API_KEY,
        'steamid': steam_id,
        'format': 'json',
        'include_appinfo': False,
        'include_played_free_games': True
    }
    try:
        response = requests.get(url, params=params, timeout=5)
        data = response.json()

        if 'response' in data and 'games' in data['response']:
            games = data['response']['games']
            owned_game_ids = [game['appid'] for game in games]
            return owned_game_ids
        else:
            return []
    except Exception as e:
        print(f"Error fetching games for Steam ID {steam_id}: {e}")
        return []


In [72]:
# Before: Build mapping once
appid_to_game_name = dict(zip(data['game_appid'], data['game_name']))

# Recommend
steam_id = 76561198030330010
recommendations = recommend_games_by_steam_id(steam_id, user_id_map, train_data, top_k=10)

# Print results nicely
print(f"\nTop 5 recommendations for Steam ID {steam_id}:")
for rank, (game_id, score) in enumerate(recommendations, start=1):
    game_name = appid_to_game_name.get(game_id, f"Unknown Game ({game_id})")
    print(f"Rank {rank}: {game_name} (Score: {score:.4f})")


Steam ID 76561198030330010 not found. Trying Steam API cold-start fallback...
Created cold-start embedding from 1206 owned games ✅

Top 5 recommendations for Steam ID 76561198030330010:
Rank 1: Wasteland Angel (Score: 1.0000)
Rank 2: TrackMania Nations Forever (Score: 1.0000)
Rank 3: Path of Exile (Score: 1.0000)
Rank 4: Batman: Arkham Origins (Score: 1.0000)
Rank 5: Trove (Score: 1.0000)
Rank 6: Unturned (Score: 1.0000)
Rank 7: Heroes & Generals (Score: 1.0000)
Rank 8: DRAGON BALL XENOVERSE (Score: 1.0000)
Rank 9: Batman: Arkham Knight (Score: 1.0000)
Rank 10: VRChat (Score: 1.0000)


In [2]:
from torch_geometric.data import HeteroData

data = HeteroData()

# data['paper'].x = ... # [num_papers, num_features_paper]
# data['author'].x = ... # [num_authors, num_features_author]
# data['institution'].x = ... # [num_institutions, num_features_institution]
# data['field_of_study'].x = ... # [num_field, num_features_field]

# data['paper', 'cites', 'paper'].edge_index = ... # [2, num_edges_cites]
# data['author', 'writes', 'paper'].edge_index = ... # [2, num_edges_writes]
# data['author', 'affiliated_with', 'institution'].edge_index = ... # [2, num_edges_affiliated]
# data['paper', 'has_topic', 'field_of_study'].edge_index = ... # [2, num_edges_topic]

# data['paper', 'cites', 'paper'].edge_attr = ... # [num_edges_cites, num_features_cites]
# data['author', 'writes', 'paper'].edge_attr = ... # [num_edges_writes, num_features_writes]
# data['author', 'affiliated_with', 'institution'].edge_attr = ... # [num_edges_affiliated, num_features_affiliated]
# data['paper', 'has_topic', 'field_of_study'].edge_attr = ... # [num_edges_topic, num_features_topic]

In [12]:
# from torch_geometric.datasets import OGB_MAG

# dataset = OGB_MAG(root='./data', preprocess='metapath2vec')
# data = dataset[0]

In [9]:
data = pd.read_csv('final_fantasy.csv')

In [19]:
from sklearn.preprocessing import LabelEncoder

user_encoder, game_encoder, genre_encoder, category_encoder = LabelEncoder(), LabelEncoder(), LabelEncoder(), LabelEncoder()

In [20]:
smol_df = data[:20000]

In [21]:
smaller_df = smol_df[['user_steam_id', 'game_appid', 'genres_en', 'categories_en', 'user_playtime_forever']]

smaller_df['user_id'] = user_encoder.fit_transform(smaller_df['user_steam_id'])
smaller_df['game_id'] = game_encoder.fit_transform(smaller_df['game_appid'])
smaller_df.head()

/var/folders/52/pkkzdswd4vv_f_f5lz_pny4m0000gn/T/ipykernel_18645/1601408051.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller_df['user_id'] = user_encoder.fit_transform(smaller_df['user_steam_id'])
/var/folders/52/pkkzdswd4vv_f_f5lz_pny4m0000gn/T/ipykernel_18645/1601408051.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller_df['game_id'] = game_encoder.fit_transform(smaller_df['game_appid'])


,user_steam_id,game_appid,genres_en,categories_en,user_playtime_forever,user_id,game_id
0,76561198060785055,240,['Action'],"['Multi-Player', 'Cross-Platform Multiplayer',...",7279,25,10
1,76561198060785055,12210,"['Action', 'Adventure']","['Single-Player', 'Multi-Player', 'Partial Con...",0,25,175
2,76561198060785055,620,"['Action', 'Adventure']","['Single Person', 'Multiple People', 'Cooperat...",717,25,18
3,76561198060785055,105600,"['Action', 'Adventure', 'Indie', 'Rpg']","['Single-Player', 'Multi-Player', 'Pvp', 'Onli...",4578,25,552
4,76561198060785055,46520,"['Action', 'Indie']","['Single-Player', 'Steam Achievements', 'Parti...",0,25,420


In [22]:
genre_map = dict()
for genres in data['genres_en']:
    if genres == 'Nan':
        continue
    for genre in eval(genres):
        if genre not in genre_map:
            genre_map[genre] = len(genre_map)



In [23]:
def genre_mapper(row):
    if row == 'Nan':
        return []
    row = eval(row)
    formatted_row = [genre_map[item] for item in row]
    return formatted_row

In [24]:
smaller_df['genre_ids'] = smaller_df['genres_en'].apply(genre_mapper).copy()
smaller_df.head()

/var/folders/52/pkkzdswd4vv_f_f5lz_pny4m0000gn/T/ipykernel_18645/3892077666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller_df['genre_ids'] = smaller_df['genres_en'].apply(genre_mapper).copy()


,user_steam_id,game_appid,genres_en,categories_en,user_playtime_forever,user_id,game_id,genre_ids
0,76561198060785055,240,['Action'],"['Multi-Player', 'Cross-Platform Multiplayer',...",7279,25,10,[0]
1,76561198060785055,12210,"['Action', 'Adventure']","['Single-Player', 'Multi-Player', 'Partial Con...",0,25,175,"[0, 1]"
2,76561198060785055,620,"['Action', 'Adventure']","['Single Person', 'Multiple People', 'Cooperat...",717,25,18,"[0, 1]"
3,76561198060785055,105600,"['Action', 'Adventure', 'Indie', 'Rpg']","['Single-Player', 'Multi-Player', 'Pvp', 'Onli...",4578,25,552,"[0, 1, 2, 3]"
4,76561198060785055,46520,"['Action', 'Indie']","['Single-Player', 'Steam Achievements', 'Parti...",0,25,420,"[0, 2]"


In [25]:
category_map = dict()
for categories in data['categories_en']:
    if categories == 'Nan':
        continue
    categories = categories.replace('s\' ', 's') # Screw you 'Players' Battles'
    for category in eval(categories):
        if category not in category_map:
            category_map[category] = len(category_map)

<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequen

In [26]:
def category_mapper(row):
    if row == 'Nan':
        return []
    row = eval(row)
    formatted_row = [category_map[item] for item in row]
    return formatted_row

In [27]:
smaller_df['categories_en'] = smaller_df['categories_en'].apply(lambda x: x.replace('s\' ', 's'))
smaller_df['category_ids'] = smaller_df['categories_en'].apply(category_mapper).copy()
smaller_df.head()

/var/folders/52/pkkzdswd4vv_f_f5lz_pny4m0000gn/T/ipykernel_18645/112053627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller_df['categories_en'] = smaller_df['categories_en'].apply(lambda x: x.replace('s\' ', 's'))
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: invalid escape sequence '\X'
<string>:1: SyntaxWarning: 

,user_steam_id,game_appid,genres_en,categories_en,user_playtime_forever,user_id,game_id,genre_ids,category_ids
0,76561198060785055,240,['Action'],"['Multi-Player', 'Cross-Platform Multiplayer',...",7279,25,10,[0],"[0, 1, 2, 3, 4, 5, 6, 7]"
1,76561198060785055,12210,"['Action', 'Adventure']","['Single-Player', 'Multi-Player', 'Partial Con...",0,25,175,"[0, 1]","[8, 0, 9, 10, 11]"
2,76561198060785055,620,"['Action', 'Adventure']","['Single Person', 'Multiple People', 'Cooperat...",717,25,18,"[0, 1]","[12, 13, 14, 15, 16, 17, 2, 18, 19, 20, 21, 21..."
3,76561198060785055,105600,"['Action', 'Adventure', 'Indie', 'Rpg']","['Single-Player', 'Multi-Player', 'Pvp', 'Onli...",4578,25,552,"[0, 1, 2, 3]","[8, 0, 29, 30, 31, 32, 2, 33, 34, 3, 10, 11, 3..."
4,76561198060785055,46520,"['Action', 'Indie']","['Single-Player', 'Steam Achievements', 'Parti...",0,25,420,"[0, 2]","[8, 2, 9, 36, 7]"


In [28]:
from torch_geometric.data import HeteroData

h_data = HeteroData() # The h is for hentai
h_data['user'].x = torch.arange(len(user_encoder.classes_))
h_data['game'].x = torch.arange(len(game_encoder.classes_))
h_data['genre'].x = torch.arange(len(genre_map))
h_data['category'].x = torch.arange(len(category_map))

In [48]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

def embed_genre_texts(texts, batch_size=32):
    all_embeddings = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True)
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # [CLS] token
            all_embeddings.append(cls_embeddings)
    return torch.cat(all_embeddings, dim=0)

In [52]:
genre_embeddings = embed_genre_texts(smaller_df['genres_en'].tolist())  # [num_rows, 768]
projector = torch.nn.Linear(768, 64)
genre_embeddings_projected = projector(genre_embeddings)  # [num_rows, 64]

In [53]:
playtime = torch.tensor(smaller_df['user_playtime_forever'].values, dtype=torch.float)

In [54]:
playtime_norm = (playtime - playtime.min()) / (playtime.max() - playtime.min() + 1e-8)

In [55]:
user_game_edges = torch.tensor([
    smaller_df['user_id'].values,
    smaller_df['game_id'].values
], dtype=torch.long)
h_data['user', 'plays', 'game'].edge_index = user_game_edges 
h_data['user', 'plays', 'game'].edge_attr = playtime_norm

In [56]:
# game → genre
game_genre_pairs = [(row['game_id'], genre_id) for _, row in smaller_df.iterrows()
    for genre_id in row['genre_ids']
]
game_genre_edges = torch.tensor(game_genre_pairs, dtype=torch.long).t()
h_data['game', 'has_genre', 'genre'].edge_index = game_genre_edges
h_data['game', 'has_genre', 'genre'].edge_attr = genre_embeddings_projected
h_data['game', 'has_genre', 'genre']

{'edge_index': tensor([[ 10, 175, 175,  ..., 344, 344, 344],
        [  0,   0,   1,  ...,   1,  10,   3]]), 'edge_attr': tensor([[-0.0287, -0.3403, -0.4023,  ..., -0.1077, -0.2097, -0.2118],
        [ 0.0809, -0.2777, -0.3392,  ..., -0.0741, -0.3333, -0.2213],
        [ 0.0809, -0.2777, -0.3392,  ..., -0.0741, -0.3333, -0.2213],
        ...,
        [ 0.1501, -0.2527, -0.2365,  ..., -0.0233, -0.3034, -0.1577],
        [-0.2195, -0.4303, -0.4682,  ..., -0.1871, -0.0988, -0.3249],
        [ 0.1121, -0.3515, -0.3555,  ..., -0.0830, -0.4086, -0.2157]],
       grad_fn=<AddmmBackward0>)}

In [57]:
# game → category
game_cat_pairs = [
    (row['game_id'], cat_id)
    for _, row in smaller_df.iterrows()
    for cat_id in row['category_ids']
]
game_cat_edges = torch.tensor(game_cat_pairs, dtype=torch.long).t()
h_data['game', 'has_category', 'category'].edge_index = game_cat_edges
h_data['game', 'has_category', 'category']

{'edge_index': tensor([[ 10,  10,  10,  ..., 344, 344, 344],
        [  0,   1,   2,  ...,  35,  41,   7]])}

In [58]:
from torch_geometric.transforms import ToUndirected
from torch_geometric.nn import HeteroConv, SAGEConv
import torch.nn as nn
h_data = ToUndirected()(h_data)
h_data['game', 'rev_plays', 'user']

{'edge_index': tensor([[  10,  175,   18,  ..., 2519, 4724,  344],
        [  25,   25,   25,  ...,   59,   59,   59]]), 'edge_attr': tensor([0.0107, 0.0000, 0.0011,  ..., 0.0000, 0.0000, 0.0000])}

In [59]:
# new architecture, different convolution approach singe SAGEConv does not support edge attributes
from torch_geometric.nn import HeteroConv, NNConv
import torch.nn.functional as F
from torch import nn

class GNN(torch.nn.Module):
    def __init__(self, num_users, num_games, num_genres, num_categories, embedding_dimensions=64):
        super().__init__()
        self.user_embed = nn.Embedding(num_users, embedding_dimensions)
        self.game_embed = nn.Embedding(num_games, embedding_dimensions)
        self.genre_embed = nn.Embedding(num_genres, embedding_dimensions)
        self.category_embed = nn.Embedding(num_categories, embedding_dimensions)

        # Edge MLP for edge attributes
        def edge_nn():
            return nn.Sequential(
                nn.Linear(1, embedding_dimensions),
                nn.ReLU(),
                nn.Linear(embedding_dimensions, embedding_dimensions * embedding_dimensions)  # critical fix
        )
        self.convs = nn.ModuleList([
            HeteroConv({
                ('user', 'plays', 'game'): NNConv(
                    in_channels=(embedding_dimensions, embedding_dimensions),
                    out_channels=embedding_dimensions,
                    nn=edge_nn()
                ),
                ('game', 'has_genre', 'genre'): NNConv(
                    (embedding_dimensions, embedding_dimensions),
                    embedding_dimensions,
                    nn=edge_nn()
                ),
                ('game', 'has_category', 'category'): NNConv(
                    (embedding_dimensions, embedding_dimensions),
                    embedding_dimensions,
                    nn=edge_nn()
                ),
                ('game', 'rev_plays', 'user'): NNConv(
                    (embedding_dimensions, embedding_dimensions),
                    embedding_dimensions,
                    nn=edge_nn()
                ),
                ('genre', 'rev_has_genre', 'game'): NNConv(
                    (embedding_dimensions, embedding_dimensions),
                    embedding_dimensions,
                    nn=edge_nn()
                ),
                ('category', 'rev_has_category', 'game'): NNConv(
                    (embedding_dimensions, embedding_dimensions),
                    embedding_dimensions,
                    nn=edge_nn()
                ),
            }, aggr='sum')
        ])

    def forward(self, x_dict, edge_index_dict, edge_attr_dict):
        x_dict = {
            'user': self.user_embed(x_dict['user']),
            'game': self.game_embed(x_dict['game']),
            'genre': self.genre_embed(x_dict['genre']),
            'category': self.category_embed(x_dict['category']),
        }
        x_dict = self.convs[0](x_dict, edge_index_dict, edge_attr_dict)
        return x_dict
    
#same LP as before
class LinkPredictor(torch.nn.Module):
    def __init__(self, embedding_dimensions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2 * embedding_dimensions, embedding_dimensions),
            nn.ReLU(),
            nn.Linear(embedding_dimensions, 1)
        )
    def forward(self, i, j):
        X = torch.cat([i,j], dim=-1)
        return torch.sigmoid(self.net(X)).squeeze()

In [60]:
# # old architecture with SAGECovn
# class GNN(torch.nn.Module):
#     def __init__(self, num_users, num_games, num_genres, num_categories, 
#                  embedding_dimensions = 64):
#         super().__init__()
#         self.user_embed = nn.Embedding(num_users, embedding_dimensions)
#         self.game_embed = nn.Embedding(num_games, embedding_dimensions)
#         self.genre_embed = nn.Embedding(num_genres, embedding_dimensions)
#         self.category_embed = nn.Embedding(num_categories, embedding_dimensions)
#         self.convs = nn.ModuleList([
#             HeteroConv({
#                 ('user', 'plays', 'game'): SAGEConv((-1, -1), embedding_dimensions),
#                 ('game', 'has_genre', 'genre'): SAGEConv((-1, -1), embedding_dimensions),
#                 ('game', 'has_category', 'category'): SAGEConv((-1, -1), embedding_dimensions),
#                 ('game', 'rev_plays', 'user'): SAGEConv((-1, -1), embedding_dimensions),
#                 ('genre', 'rev_has_genre', 'game'): SAGEConv((-1, -1), embedding_dimensions),
#                 ('category', 'rev_has_category', 'game'): SAGEConv((-1, -1), embedding_dimensions),
#             }, aggr='sum')
#         ])

#     def forward(self, x_dict, edge_index_dict):
#         x_dict = {
#             'user': self.user_embed(x_dict['user']),
#             'game': self.game_embed(x_dict['game']),
#             'genre': self.genre_embed(x_dict['genre']),
#             'category': self.category_embed(x_dict['category']),
#         }
#         x_dict = self.convs[0](x_dict, edge_index_dict)  # Access first layer in the list
#         return x_dict

# class LinkPredictor(torch.nn.Module):
#     def __init__(self, embedding_dimensions):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(2 * embedding_dimensions, embedding_dimensions),
#             nn.ReLU(),
#             nn.Linear(embedding_dimensions, 1)
#         )
#     def forward(self, i, j):
#         X = torch.cat([i,j], dim=-1)
#         return torch.sigmoid(self.net(X)).squeeze()

In [61]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(
    num_users=h_data['user'].num_nodes,
    num_games=h_data['game'].num_nodes,
    num_genres=h_data['genre'].num_nodes,
    num_categories=h_data['category'].num_nodes,
    embedding_dimensions=64
).to(device)

In [62]:
import random
predictor = LinkPredictor(64).to(device)

optimizer = torch.optim.Adam(list(model.parameters()) + list(predictor.parameters()), lr=0.001)
h_data = h_data.to(device)

# === Create Positive and Negative Edge Pairs ===
positive_edges = h_data['user', 'plays', 'game'].edge_index.t().tolist()

def sample_batch(batch_size=1024):
    pos_samples = random.sample(positive_edges, batch_size)
    neg_samples = []
    user_nodes = h_data['user'].x.tolist()
    game_nodes = h_data['game'].x.tolist()

    while len(neg_samples) < batch_size:
        u = random.choice(user_nodes)
        g = random.choice(game_nodes)
        if [u, g] not in positive_edges:
            neg_samples.append([u, g])
    return pos_samples, neg_samples

In [63]:
edge_attr_dict = {}

for edge_type in h_data.edge_types:
    ei = h_data[edge_type].edge_index
    if 'edge_attr' in h_data[edge_type]:
        edge_attr_dict[edge_type] = h_data[edge_type].edge_attr
    else:
        edge_attr_dict[edge_type] = torch.ones(ei.shape[1], 1)  # dummy edge features
edge_attr_dict

{('user',
  'plays',
  'game'): tensor([0.0107, 0.0000, 0.0011,  ..., 0.0000, 0.0000, 0.0000]),
 ('game',
  'has_genre',
  'genre'): tensor([[-0.0287, -0.3403, -0.4023,  ..., -0.1077, -0.2097, -0.2118],
         [ 0.0809, -0.2777, -0.3392,  ..., -0.0741, -0.3333, -0.2213],
         [ 0.0809, -0.2777, -0.3392,  ..., -0.0741, -0.3333, -0.2213],
         ...,
         [ 0.1501, -0.2527, -0.2365,  ..., -0.0233, -0.3034, -0.1577],
         [-0.2195, -0.4303, -0.4682,  ..., -0.1871, -0.0988, -0.3249],
         [ 0.1121, -0.3515, -0.3555,  ..., -0.0830, -0.4086, -0.2157]],
        grad_fn=<AddmmBackward0>),
 ('game',
  'has_category',
  'category'): tensor([[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]),
 ('game',
  'rev_plays',
  'user'): tensor([0.0107, 0.0000, 0.0011,  ..., 0.0000, 0.0000, 0.0000]),
 ('genre',
  'rev_has_genre',
  'game'): tensor([[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]])

In [64]:
h_data['user', 'plays', 'game'].edge_attr

tensor([0.0107, 0.0000, 0.0011,  ..., 0.0000, 0.0000, 0.0000])

In [65]:
edge_attr_dict = {
    ('user', 'plays', 'game'): h_data['user', 'plays', 'game'].edge_attr.view(-1, 1),
    ('game', 'has_genre', 'genre'): h_data['game', 'has_genre', 'genre'].edge_attr.view(-1,1),
    ('game', 'has_category', 'category'): torch.ones(h_data['game', 'has_category', 'category'].edge_index.size(1), 1).to(device),
    ('game', 'rev_plays', 'user'): h_data['user', 'plays', 'game'].edge_attr.view(-1, 1),
    ('genre', 'rev_has_genre', 'game'): torch.ones(h_data['genre', 'rev_has_genre', 'game'].edge_index.size(1), 1).to(device),
    ('category', 'rev_has_category', 'game'): torch.ones(h_data['category', 'rev_has_category', 'game'].edge_index.size(1), 1).to(device),
}

In [66]:
# edge_attr_dict = {
#     ('user', 'plays', 'game'): h_data['user', 'plays', 'game'].edge_attr.view(-1, 1),
#     ('game', 'has_genre', 'genre'): torch.ones(h_data['game', 'has_genre', 'genre'].edge_index.size(1), 1).to(device),
#     ('game', 'has_category', 'category'): torch.ones(h_data['game', 'has_category', 'category'].edge_index.size(1), 1).to(device),
#     ('game', 'rev_plays', 'user'): h_data['user', 'plays', 'game'].edge_attr.view(-1, 1),
#     ('genre', 'rev_has_genre', 'game'): torch.ones(h_data['genre', 'rev_has_genre', 'game'].edge_index.size(1), 1).to(device),
#     ('category', 'rev_has_category', 'game'): torch.ones(h_data['category', 'rev_has_category', 'game'].edge_index.size(1), 1).to(device),
# }

In [67]:
for k, v in edge_attr_dict.items():
    print(f"{k}: {v.shape}")

('user', 'plays', 'game'): torch.Size([20000, 1])
('game', 'has_genre', 'genre'): torch.Size([1280000, 1])
('game', 'has_category', 'category'): torch.Size([161093, 1])
('game', 'rev_plays', 'user'): torch.Size([20000, 1])
('genre', 'rev_has_genre', 'game'): torch.Size([50712, 1])
('category', 'rev_has_category', 'game'): torch.Size([161093, 1])


In [68]:
# new
from tqdm import tqdm
from torch.nn.functional import binary_cross_entropy

epochs = 50
x_dict = model(h_data.x_dict, h_data.edge_index_dict, edge_attr_dict)

# Wrap epochs in tqdm for progress tracking
for epoch in tqdm(range(epochs), desc="Training Progress", unit="epoch"):
    model.train()
    predictor.train()
    
    pos_edges, neg_edges = sample_batch(batch_size=256)

    # Combine and format edges
    edges = torch.tensor(pos_edges + neg_edges, dtype=torch.long).to(device)
    if edges.ndim == 1:
        edges = edges.view(-1, 2)

    labels = torch.cat([
        torch.ones(len(pos_edges)),
        torch.zeros(len(neg_edges))
    ]).to(device)

    users = edges[:, 0]
    games = edges[:, 1]

    # Forward pass
    x_dict = model(h_data.x_dict, h_data.edge_index_dict, edge_attr_dict)
    user_emb = x_dict['user'][users]
    game_emb = x_dict['game'][games]
    preds = predictor(user_emb, game_emb)

    # Loss
    loss = binary_cross_entropy(preds, labels)

    # Accuracy
    pred_labels = (preds >= 0.5).float()
    acc = (pred_labels == labels).float().mean()

    # Optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1:02d} | Loss: {loss.item():.4f} | Accuracy: {acc.item():.4f}")

KeyboardInterrupt: 

In [ ]:
# from tqdm import tqdm
# from torch.nn.functional import binary_cross_entropy

# epochs = 50
# x_dict = model(
#     h_data.x_dict,
#     h_data.edge_index_dict,
#     edge_attr_dict={
#         ('user', 'plays', 'game'): h_data['user', 'plays', 'game'].edge_attr,
#         ('game', 'has_genre', 'genre'): torch.ones(h_data['game', 'has_genre', 'genre'].edge_index.size(1), 1),
#         ('game', 'has_category', 'category'): torch.ones(h_data['game', 'has_category', 'category'].edge_index.size(1), 1),
#         ('game', 'rev_plays', 'user'): h_data['user', 'plays', 'game'].edge_attr,  # mirror
#         ('genre', 'rev_has_genre', 'game'): torch.ones(h_data['genre', 'rev_has_genre', 'game'].edge_index.size(1), 1),
#         ('category', 'rev_has_category', 'game'): torch.ones(h_data['category', 'rev_has_category', 'game'].edge_index.size(1), 1),
#     }
# )

# # Wrap epochs in tqdm for progress tracking
# for epoch in tqdm(range(epochs), desc="Training Progress", unit="epoch"):
#     model.train()
#     predictor.train()
    
#     pos_edges, neg_edges = sample_batch(batch_size=4096)
#     edges = pos_edges + neg_edges
#     labels = torch.cat([
#         torch.ones(len(pos_edges)),
#         torch.zeros(len(neg_edges))
#     ]).to(device)


#     edges = torch.tensor(edges, dtype=torch.long).to(device)
    
#     users = edges[:, 0]
#     games = edges[:, 1]

#     # Forward pass
#     x_dict = model(h_data.x_dict, h_data.edge_index_dict)
#     user_emb = x_dict['user'][users]
#     game_emb = x_dict['game'][games]
#     preds = predictor(user_emb, game_emb)

#     # Loss
#     loss = binary_cross_entropy(preds, labels)

#     # Accuracy
#     pred_labels = (preds >= 0.5).float()
#     acc = (pred_labels == labels).float().mean()

#     # Optimize
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     print(f"Epoch {epoch+1:02d} | Loss: {loss.item():.4f} | Accuracy: {acc.item():.4f}")

### Save & Load

In [152]:
# torch.save(model.state_dict(), "gnn_model.pth")
# torch.save(predictor.state_dict(), "link_predictor.pth")

model.load_state_dict(torch.load("gnn_model.pth"))
predictor.load_state_dict(torch.load("link_predictor.pth"))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
x_dict = model(h_data.x_dict, h_data.edge_index_dict)

users = torch.tensor(smaller_df['user_id'], dtype=torch.long).to(device)
games = torch.tensor(smaller_df['game_id'], dtype=torch.long).to(device)

user_emb = x_dict['user'][users]
game_emb = x_dict['game'][users]

preds = predictor(user_emb, game_emb)

pred_labels = (preds >= .5).float()

In [ ]:
pd.Series(pred_labels.tolist()).value_counts()

1.0    1050157
0.0     946256
Name: count, dtype: int64

In [ ]:
# Choose the user for which you want recommendations
user_id = torch.tensor([42], dtype=torch.long).to(device)  # Replace 42 with your target user ID

# Get all unique game IDs
all_games = torch.arange(h_data['game'].num_nodes, dtype=torch.long).to(device)

# Expand user tensor to match all possible games
users_expanded = user_id.repeat_interleave(all_games.shape[0])

# Forward pass through the GNN
x_dict = model(h_data.x_dict, h_data.edge_index_dict)
user_emb = x_dict['user'][users_expanded]  # Get the target user's embeddings
game_emb = x_dict['game'][all_games]  # Get all possible game embeddings

# Predict likelihood of interaction for each game
preds = predictor(user_emb, game_emb)

# Rank games by highest prediction scores
sorted_indices = torch.argsort(preds, descending=True)
recommended_games = all_games[sorted_indices]

# Display top recommendations
print(f"Top recommended games for user {user_id.item()}: {recommended_games[:10].tolist()}")

Top recommended games for user 42: [24333, 30764, 30559, 26364, 20, 27757, 21042, 29980, 1810, 26266]


{'x': tensor([    0,     1,     2,  ..., 31394, 31395, 31396], device='cuda:0')}

In [58]:
import torch
import pandas as pd

def recommend_games(user_id: int, data: pd.DataFrame, user_encoder, game_encoder, model, predictor, h_data, device, n_games: int = 5,):
    """
    Recommends the top n_games for a given user_id, converting encoded values into actual game names.

    Parameters:
    - user_id (int): Original user ID from Steam.
    - n_games (int): Number of game recommendations to return.
    - data (pd.DataFrame): Dataframe containing game and user metadata.
    - user_encoder (LabelEncoder): Fitted encoder mapping user IDs to numeric values.
    - model (torch.nn.Module): Trained GNN model.
    - predictor (torch.nn.Module): Trained link predictor.
    - h_data (HeteroData): Heterogeneous graph data with nodes and edges.
    - device (torch.device): The device (CPU/GPU) for tensor operations.

    Returns:
    - List of recommended game names.
    """

    # Convert original user ID to encoded ID
    encoded_user_id = user_encoder.transform([user_id])[0]

    # Get all unique game IDs (these correspond to `game_appid`)
    all_games = torch.arange(h_data['game'].num_nodes, dtype=torch.long).to(device)

    # Expand user tensor to match all possible games
    user_tensor = torch.tensor([encoded_user_id], dtype=torch.long).to(device)
    users_expanded = user_tensor.repeat_interleave(all_games.shape[0])

    # Forward pass through GNN
    x_dict = model(h_data.x_dict, h_data.edge_index_dict)
    user_emb = x_dict['user'][users_expanded]
    game_emb = x_dict['game'][all_games]

    # Predict interaction likelihood
    preds = predictor(user_emb, game_emb)

    # Sort games by highest predicted score
    sorted_indices = torch.argsort(preds, descending=True)
    recommended_appids = all_games[sorted_indices][:n_games].tolist()  # Get top N games
    recommended_appids = game_encoder.inverse_transform(recommended_appids)
    recommended_game_names = data[data['game_appid'].isin(recommended_appids)]['game_name']

    return recommended_game_names

In [60]:
user_id = 76561199837232708 # Jarvin's ID is: 76561198030330010 and has not been seen by the model yet.
# https://steamcommunity.com/profiles/76561198025042360/
recommend_games(user_id = user_id, data = data, user_encoder = user_encoder, game_encoder = game_encoder, model = model, predictor = predictor, h_data = h_data, device = device)

7          Counter-Strike: Global Offensive
247        Counter-Strike: Global Offensive
529        Counter-Strike: Global Offensive
728        Counter-Strike: Global Offensive
1105       Counter-Strike: Global Offensive
                         ...               
1994103    Counter-Strike: Global Offensive
1994579    Counter-Strike: Global Offensive
1995779    Counter-Strike: Global Offensive
1996041    Counter-Strike: Global Offensive
1996334    Counter-Strike: Global Offensive
Name: game_name, Length: 5378, dtype: object

In [55]:
game_encoder.inverse_transform([24333, 30559, 30764, 26364]).tolist()

[1211930, 2455340, 2581050, 1456650]

In [57]:
data[data['game_appid'].isin(game_encoder.inverse_transform([24333, 30559, 30764, 26364]).tolist())]

,Unnamed: 0,user_steam_id,user_playtime_forever,user_playtime_2weeks,user_has_leaderboards,user_has_community_visible_stats,user_content_descriptorids,game_appid,game_name,game_developer,...,game_price,game_initialprice,game_discount,game_ccu,game_owners_estimate,game_review_ratio,genres_en,categories_en,short_description_en,detailed_description_en
336460,336460,76561198067808950,307,0.0,False,True,[0],1456650,Pascal's Wager: Definitive Edition,TipsWorks,...,19.99,19.99,0.0,10,100000,0.705138,"['Action', 'Adventure', 'Rpg']","['Single-Player', 'Steam Achievements', 'Steam...",Pascal's Wager: Definitive Edition is a challe...,Special OfferPascal's Wager: Definitive Editio...
1585099,1585099,76561198010642927,163,0.0,False,True,[0],1211930,Ghost on the Shore,like Charlie,...,19.99,19.99,0.0,2,0,0.840426,['Adventure'],"['Single-Player', 'Steam Achievements', 'Full ...",An exploration game about emotional ties that ...,FEATURESDiscover the branching narrative and m...
1688785,1688785,76561198106338688,201,0.0,False,True,[0],2455340,Together in Between,Vellichor Games,...,6.99,6.99,0.0,4,0,0.632302,"['Casual', 'Indie']","['Multi-Player', 'Co-Op', 'Online Co-Op', 'Ste...",Together in Between is a first-person adventur...,"Welcome to ""Together in Between,"" a unique co-..."
1731023,1731023,76561198061375313,767,30.0,False,True,[0],2581050,KAMITSUBAKI CITY ENSEMBLE,"KAMITSUBAKI STUDIO, Studio Lalala",...,3.99,3.99,0.0,22,20000,0.689469,"['Action', 'Adventure', 'Casual', 'Indie', 'Sp...","['Single-Player', 'Steam Achievements', 'Parti...","""KAMITSUBAKI CITY ENSEMBLE"" is a rhythm game i...","The new rhythm game, ""KAMITSUBAKI CITY ENSEMBL..."
